In [15]:
import lancedb
import pyarrow as pa
from datasets import load_dataset
import pandas as pd
import numpy as np
from model import EncoderModel, DecoderModel, BM25Model
from tqdm import tqdm
from model import EncoderModel
import torch

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "sentence-transformers/all-MiniLM-L6-v2"
encoder = EncoderModel(model_id, device)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [17]:
sentences = ["Hello World", "Hello Mars", "Transformers are awesome"]
vectors = encoder(sentences)

In [18]:
vectors.shape

(3, 384)

In [19]:
collection = lancedb.connect("./db")
custom_schema = pa.schema([
    pa.field("vector", pa.list_(pa.float32(), vectors.shape[-1])),
    pa.field("text", pa.string())
])
table = collection.create_table("test", schema=custom_schema)

OSError: LanceError(IO): Generic LocalFileSystem error: Unable to copy file from /home/chkei001/abgabe/db/test.lance/_versions/1.manifest-378a64b5-c0a7-497d-aa70-0908dbfec013 to /home/chkei001/abgabe/db/test.lance/_latest.manifest: Device or resource busy (os error 16), /home/runner/work/lance/lance/rust/lance-table/src/io/commit.rs:180:5

In [21]:
table = collection.open_table("test")

In [26]:
add_docs = np.vectorize(lambda text, vector: table.add([{"text": text, "vector": vector}]))

In [27]:
add_docs(sentences, vectors)

TypeError: Unsupported vector column type: float

In [31]:
vectors[0].dtype

dtype('float32')

In [29]:
table.schema

vector: fixed_size_list<item: float>[384]
  child 0, item: float
text: string

In [34]:
for t, v in zip(sentences, vectors):
    table.add([
        {
            "vector": v,
            "text": t
        }
    ])

OSError: LanceError(IO): Generic LocalFileSystem error: Unable to copy file from /home/chkei001/abgabe/db/test.lance/_versions/3.manifest-a91be8c1-dea5-4f96-9713-1db44a8a0ff4 to /home/chkei001/abgabe/db/test.lance/_latest.manifest: Device or resource busy (os error 16), /home/runner/work/lance/lance/rust/lance-table/src/io/commit.rs:180:5

In [35]:
table.to_pandas()

,vector,text
0,"[-0.03447729, 0.031023229, 0.0067349756, 0.026...",Hello World
